In [ ]:
import ee
import geemap
Map = geemap.Map(center=[34.797058518162295, 114.30106209854621], zoom=8)
#Map.add_basemap('ROADMAP') # Add Google Map
Map


In [ ]:
Map.setCenter(114.2,34.79,  9)
geometry = ee.Geometry.Rectangle([114.29,34.79, 114.31, 34.81])
    
# Create a source image where the geometry is 1, everything else is 0.
sources = ee.Image().toByte().paint(geometry, 1)

# Mask the sources image with itself.
sources = sources.updateMask(sources)

cover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(0)

Map.addLayer(cover, {}, 'Globcover')
Map.addLayer(geometry, {'color': 'FF0000'}, 'source geometry')


In [ ]:
# The cost data is generated from classes in ESA/GLOBCOVER.


# Classes 60, 80, 110, 140 have cost 1.
# Classes 40, 90, 120, 130, 170 have cost 2.
# Classes 50, 70, 150, 160 have cost 3.
cost = \
  cover.eq(14).Or(cover.eq(80)).Or(cover.eq(110)).Or(cover.eq(140)) \
      .multiply(1).add(
  cover.eq(11).Or(cover.eq(90)).Or(cover.eq(120)).Or(cover.eq(130)) \
    .Or(cover.eq(170)) \
      .multiply(2).add(
  cover.eq(50).Or(cover.eq(70)).Or(cover.eq(150)).Or(cover.eq(190)) \
      .multiply(5)))
# Compute the cumulative cost to traverse the lAnd cover.
cumulativeCost = cost.cumulativeCost(**{
  'source': sources,
  'maxDistance': 50 * 1000 # 80 kilometers
})


Map.addLayer(cumulativeCost, {'min': 0, 'max': 5e4}, 'accumulated cost')
